# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [2]:
# Mounting and extracting content
#from google.colab import drive
#drive.mount('/content/drive')
#! cp "/content/drive/Othercomputers/My Mac mini/MachineLearning/ml-intro/1_data_manipulations/database.sqlite" ./

In [3]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [4]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)
player_attributes = pd.read_sql("SELECT * FROM Player_Attributes;", db)
league_data = pd.read_sql("SELECT * FROM League;", db)
match_data = pd.read_sql("SELECT * FROM Match;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)

In [5]:
player_data['birthday_year'] = pd.to_datetime(player_data.birthday).dt.year

In [6]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,birthday_year
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,1992
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146,1989
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163,1991
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198,1982
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154,1979


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [23]:
players_180_190 = (
      (player_data.height <= 190) & (player_data.height >= 180)
    ).sum()
players_180_190

5925

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [24]:
players_1980 = (player_data.birthday_year == 1980).sum()
players_1980

434

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [26]:
highest_players = player_data.sort_values(['weight', 'player_name'], ascending=[False, True]) \
    .head(10).player_name.values.tolist()

highest_players

['Kristof van Hout',
 'Tim Wiese',
 'Jeroen Verhoeven',
 'Ishmael Miller',
 'Cassio',
 'Christopher Samba',
 'Lars Unnerstall',
 'Marcus Hahnemann',
 'Abdoulaye Faye',
 'Enoch Showunmi']

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [27]:
years_born_players = list(
    player_data[(player_data.birthday_year >= 1980) & (player_data.birthday_year <= 1990)]
      .birthday_year
      .value_counts()
      .sort_index()
      .items()
    )

years_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [28]:
adriano_mean, adriano_std = (
    player_data
      .loc[
        player_data.player_name.str.split(" ").str[0] == "Adriano",
        'height'
      ]
      .describe()[['mean','std']]
    )

print(f"adriano_mean={adriano_mean}, adriano_std={adriano_std}")

adriano_mean=182.11800000000002, adriano_std=5.361479480723788


**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [29]:
dow_with_min_players_born = (
      pd.to_datetime(player_data.birthday)
        .dt.day_name()
        .value_counts()
        .sort_values()
        .index[0]
    )

dow_with_min_players_born

'Sunday'

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [30]:
league_most_matches = match_data.league_id.value_counts().rename('matches').to_frame() \
.join(league_data.set_index('id')).sort_values(['matches', 'name'], ascending=[False, True]).iloc[0,2]

league_most_matches

'England Premier League'

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [31]:
max_matches_player = pd.merge(
  match_data[
    [f'home_player_{position}' for position in range(1,12)] 
    + [f'away_player_{position}' for position in range(1,12)]
  ]
    .stack()
    .value_counts()
    .rename('matches_played')
    .head(1)
    .to_frame(),
  player_data,
  left_index=True,
  right_on='player_api_id'
).iloc[0,3]

max_matches_player

'Steve Mandanda'

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [15]:
excluded_columns = ['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']
player_columns = [column for column in player_attributes.columns if column not in excluded_columns]

corr = (
    player_attributes[player_columns]
      .corr().abs()
      .unstack()
      .sort_values(ascending=False)
      .to_frame()
      .reset_index()
      .set_axis(['col_1', 'col_2', 'corr'], axis='columns')
)

most_correlated = (
    corr[corr.col_1 != corr.col_2]
      .apply(lambda x: sorted([x['col_1'], x['col_2']]) + [x['corr']], axis=1, result_type='expand')
      .set_axis(['col_1', 'col_2', 'corr'], axis='columns')
      .drop_duplicates()
)

top_correlated_features = list(
    most_correlated[['col_1', 'col_2']]
    .head(5)
    .itertuples(index=False, name=None)
)

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same <br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [41]:
excluded_columns = ['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', \
                    'defensive_work_rate']
player_columns = [column for column in player_attributes.columns if column not in excluded_columns]

neymar_id = player_data[player_data.player_name.str.startswith("Neymar")].iloc[0, 1]

fresh_player_attributes = (
  player_attributes
    .dropna(subset=player_columns)
    .sort_values(by='date', ascending=False)
    .drop_duplicates(subset='player_api_id')
    .set_index('player_api_id')
    [player_columns]
)

neymar_attributes = fresh_player_attributes.loc[neymar_id]
other_players_attributes = fresh_player_attributes[fresh_player_attributes.index != neymar_id]

print(type(neymar_attributes))
print(type(other_players_attributes))

quad_eudist = ((other_players_attributes - neymar_attributes) ** 2).sum(axis=1)
#print(quad_eudist)

neymar_similarities = (
  pd.merge(
    quad_eudist
      .rename('quad_eudist')
      .to_frame(),
    player_data,
    left_index=True,
    right_on='player_api_id'
  )
    .sort_values(by=['quad_eudist', 'player_name'])
    .head(5)
    .player_name
    .tolist()
)

neymar_similarities

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


['Paulo Dybala',
 'Eden Hazard',
 'Giovani dos Santos',
 'Arjen Robben',
 'Lionel Messi']

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [44]:
borussia_id = team_data[team_data.team_long_name == 'Borussia Dortmund'].iloc[0, 1]
bundesliga_id = league_data[league_data.name == 'Germany 1. Bundesliga'].iloc[0,0]

home_matches = match_data[(match_data.league_id == bundesliga_id) & (match_data.season == '2008/2009') \
                          & (match_data.home_team_api_id == borussia_id)]

borussia_bundesliga_2008_2009_matches = home_matches.shape[0]
borussia_bundesliga_2008_2009_matches

17

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [45]:
team_most_matches_bundesliga_2008_2009 = (
  match_data[(match_data.league_id == bundesliga_id) & (match_data.season == '2008/2009')]
    [['home_team_api_id', 'away_team_api_id']]
    .melt().value
    .value_counts()
    .iloc[0]
)
team_most_matches_bundesliga_2008_2009

34

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [46]:
arsenal_id = team_data[team_data.team_long_name == 'Arsenal'].iloc[0, 1]

arsenal_matches = (
    match_data[((match_data.home_team_api_id == arsenal_id) | (match_data.away_team_api_id == arsenal_id)) & (match_data.season == '2015/2016')]
      .loc[:,'home_team_api_id':'away_team_goal']
)

arsenal_matches['arsenal_score'] = arsenal_matches['home_team_goal'].where(arsenal_matches.home_team_api_id == arsenal_id, other=arsenal_matches['away_team_goal'])
arsenal_matches['opponent_score'] = arsenal_matches['home_team_goal'].where(arsenal_matches.home_team_api_id != arsenal_id, other=arsenal_matches['away_team_goal'])
arsenal_matches['victory'] = arsenal_matches['arsenal_score'] > arsenal_matches['opponent_score']

arsenal_won_matches_2015_2016 = arsenal_matches.victory.sum()
arsenal_won_matches_2015_2016

20

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [47]:
season_matches = match_data[match_data.season == '2015/2016'].loc[:,'home_team_api_id':'away_team_goal']
season_matches['home_win'] = season_matches.home_team_goal > season_matches.away_team_goal
season_matches['away_win'] = season_matches.away_team_goal > season_matches.home_team_goal

team_highest_winrate_2015_2016 = (
  pd.merge(
    pd.concat(
      [
        season_matches[['home_team_api_id', 'home_win']].set_axis(['team_id', 'victory'], axis=1),
        season_matches[['away_team_api_id', 'away_win']].set_axis(['team_id', 'victory'], axis=1)
      ],
      ignore_index=True
    )
      .groupby('team_id')
      .mean(),
    team_data,
    left_index=True,
    right_on='team_api_id'
  )
    .sort_values(['victory','team_long_name'], ascending=[False, True])
    .team_long_name
    .iloc[0]
)

team_highest_winrate_2015_2016

'SL Benfica'

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [48]:
england_premier_id = league_data[league_data.name == 'England Premier League'].iloc[0,0]

england_premier_season_matches = (
    match_data[(match_data.season == '2010/2011') & (match_data.league_id == england_premier_id)]
      [['date', 'home_team_api_id', 'away_team_api_id']]
)

team_matches = (
  pd.concat(
    [
      england_premier_season_matches[['date', 'home_team_api_id']].set_axis(['date', 'team_id'], axis=1),
      england_premier_season_matches[['date', 'away_team_api_id']].set_axis(['date', 'team_id'], axis=1)
    ],
    ignore_index=True
  )
    .sort_values('date')
)

team_matches['date'] = pd.to_datetime(team_matches['date'])

highest_gap_england_2010_2011 = team_matches.groupby('team_id').diff().date.max().days
highest_gap_england_2010_2011

18

### Warning! Do not change anything in the area below

In [22]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")